<a href="https://colab.research.google.com/github/AncalagonDBlack/source-code-seo-tool/blob/master/Semantic_Keyword_Cluster_Tool_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install chardet
!pip install detect_delimiter
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=38b49fd1ace3432f115df6c4047c2e2d14ec758d38e74b812

In [2]:
import sys
import time
import sys
import pandas as pd
import chardet
import codecs
from detect_delimiter import detect
from google.colab import files
from sentence_transformers import SentenceTransformer, util

In [28]:
# upload the keyword export
upload = files.upload()
input_file = list(upload.keys())[0]  # get the name of the uploaded file

Saving Keyword Stats 2023-01-09 at 09_51_39 - Trang tính1.csv to Keyword Stats 2023-01-09 at 09_51_39 - Trang tính1.csv


In [29]:
cluster_accuracy = 85  # 0-100 (100 = very tight clusters, but higher percentage of no_cluster groups)
min_cluster_size = 2  # set the minimum size of cluster groups. (Lower number = tighter groups)

In [30]:
#list model https://www.sbert.net/docs/pretrained_models.html#model-overview => Chọn model phù hợp với KW
transformer = 'all-mpnet-base-v2'  # provides the best quality
#transformer = 'all-MiniLM-L6-v2'  # 5 times faster and still offers good quality

In [31]:
# automatically detect the character encoding type

acceptable_confidence = .8

contents = upload[input_file]

codec_enc_mapping = {
    codecs.BOM_UTF8: 'utf-8-sig',
    codecs.BOM_UTF16: 'utf-16',
    codecs.BOM_UTF16_BE: 'utf-16-be',
    codecs.BOM_UTF16_LE: 'utf-16-le',
    codecs.BOM_UTF32: 'utf-32',
    codecs.BOM_UTF32_BE: 'utf-32-be',
    codecs.BOM_UTF32_LE: 'utf-32-le',
}

encoding_type = 'utf-8'  # Default assumption
is_unicode = False

for bom, enc in codec_enc_mapping.items():
    if contents.startswith(bom):
        encoding_type = enc
        is_unicode = True
        break

if not is_unicode:
    # Didn't find BOM, so let's try to detect the encoding
    guess = chardet.detect(contents)
    if guess['confidence'] >= acceptable_confidence:
        encoding_type = guess['encoding']

print("Character Encoding Type Detected", encoding_type)

Character Encoding Type Detected utf-8


In [32]:
# automatically detect the delimiter
with open(input_file,encoding=encoding_type) as myfile:
    firstline = myfile.readline()
myfile.close()
delimiter_type = detect(firstline)

In [33]:
# create a dataframe using the detected delimiter and encoding type
df = pd.read_csv((input_file), on_bad_lines='skip', encoding=encoding_type, delimiter=delimiter_type)
count_rows = len(df)
if count_rows > 50_000:
  print("WARNING: You May Experience Crashes When Processing Over 50,000 Keywords at Once. Please consider smaller batches!")
print("Uploaded Keyword CSV File Successfully!")
df

Uploaded Keyword CSV File Successfully!


,Keyword,Avg. monthly searches
0,nấu ăn cho trẻ,10
1,món ngon cho bé,1600
2,món ăn bổ dưỡng cho trẻ,10
3,cách nấu cháo lươn,2400
4,cách nấu cháo chim bồ câu,1300
...,...,...
1751,các món cháo bổ dưỡng cho bé ăn dặm,10
1752,các món ngon dành cho bé 3 tuổi,10
1753,cách nấu các món ngon cho bé,10
1754,mon ngon cho be 9 thang tuoi,10


In [34]:
# standardise the keyword columns
df.rename(columns={"Search term": "Keyword", "keyword": "Keyword", "query": "Keyword", "query": "Keyword", "Top queries": "Keyword", "queries": "Keyword", "Keywords": "Keyword","keywords": "Keyword", "Search terms report": "Keyword"}, inplace=True)

if "Keyword" not in df.columns:
  print("Error! Please make sure your csv file contains a column named 'Keyword!")

# store the data
cluster_name_list = []
corpus_sentences_list = []
df_all = []

corpus_set = set(df['Keyword'])
corpus_set_all = corpus_set
cluster = True

In [ ]:
#cluster_accuracy = cluster_accuracy / 100
#model = SentenceTransformer(transformer)

#corpus_sentences = list(corpus_set)
#check_len = len(corpus_sentences)
#corpus_embeddings = model.encode(corpus_sentences, batch_size=256, show_progress_bar=True, convert_to_tensor=True)
#init_max_size=len(corpus_embeddings)
#init_max_size

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

1044

In [35]:
# keep looping through until no more clusters are created

cluster_accuracy = cluster_accuracy / 100
model = SentenceTransformer(transformer)

while cluster:

    corpus_sentences = list(corpus_set)
    check_len = len(corpus_sentences)

    corpus_embeddings = model.encode(corpus_sentences, batch_size=256, show_progress_bar=True, convert_to_tensor=True)
    clusters = util.community_detection(corpus_embeddings, min_community_size=min_cluster_size, threshold=cluster_accuracy)

    for keyword, cluster in enumerate(clusters):
        print("\nCluster {}, #{} Elements ".format(keyword + 1, len(cluster)))

        for sentence_id in cluster[0:]:
            print("\t", corpus_sentences[sentence_id])
            corpus_sentences_list.append(corpus_sentences[sentence_id])
            cluster_name_list.append("Cluster {}, #{} Elements ".format(keyword + 1, len(cluster)))

    df_new = pd.DataFrame(None)
    df_new['Cluster Name'] = cluster_name_list
    df_new["Keyword"] = corpus_sentences_list

    df_all.append(df_new)
    have = set(df_new["Keyword"])

    corpus_set = corpus_set_all - have
    remaining = len(corpus_set)
    print("Total Unclustered Keywords: ", remaining)
    if check_len == remaining:
        break

Batches:   0%|          | 0/7 [00:00<?, ?it/s]


Cluster 1, #144 Elements 
	 nấu nui ngon cho bé
	 ngô ngọt nấu cháo cho bé
	 các món bánh ngon cho bé
	 cách nấu cháo tôm ngon cho bé
	 nấu canh ngon cho bé
	 các món cháo gà cho bé
	 cách nấu cháo ếch ngon nhất
	 nấu ăn ngon cho bé
	 cách nấu cháo ngon cho bé ăn dặm
	 cách nấu món cháo lươn ngon
	 nấu cơm ngon cho bé
	 các loại cháo ngon cho bé
	 cách nấu cháo thịt bò ngon cho bé
	 nhung mon ngon cho be
	 cách nấu các loại cháo cho bé
	 cách làm cơm ngon cho bé
	 cách nấu cháo chay cho bé
	 cách nấu cháo lươn ngon cho bé
	 cách nấu cháo cho bé ăn ngon miệng
	 nấu cháo lươn ngon cho bé
	 cach nau mon ngon cho be
	 cách làm món ngon cho bé
	 cách làm tôm ngon cho bé
	 cách nấu cháo ngan ngon cho bé
	 cách nấu cháo cho bé ngon miệng
	 cách nấu cháo gà ngon cho bé
	 nau an ngon cho be
	 cách nấu cháo dinh dưỡng ngon cho bé
	 cách nấu các món ăn ngon cho bé
	 cháo ngon cho bé
	 các món kho ngon cho bé
	 cách kho gà ngon cho bé
	 cách nấu cháo ngon cho con
	 các món canh ngon cho bé
	 hướn

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Total Unclustered Keywords:  65


In [36]:
# make a new dataframe from the list of dataframe and merge back into the orginal df
df_new = pd.concat(df_all)
df = df.merge(df_new.drop_duplicates('Keyword'), how='left', on="Keyword")

In [37]:
# rename the clusters to the shortest keyword in the cluster
df['Length'] = df['Keyword'].astype(str).map(len)
df = df.sort_values(by="Length", ascending=True)

df['Cluster Name'] = df.groupby('Cluster Name')['Keyword'].transform('first')
df.sort_values(['Cluster Name', "Keyword"], ascending=[True, True], inplace=True)

df['Cluster Name'] = df['Cluster Name'].fillna("zzz_no_cluster")

del df['Length']

In [38]:
# move the cluster and keyword columns to the front
col = df.pop("Keyword")
df.insert(0, col.name, col)

col = df.pop('Cluster Name')
df.insert(0, col.name, col)

df.sort_values(["Cluster Name", "Keyword"], ascending=[True, True], inplace=True)

In [39]:
uncluster_percent = (remaining / count_rows) * 100
clustered_percent = 100 - uncluster_percent
print(clustered_percent,"% of rows clustered successfully!")

96.2984054669704 % of rows clustered successfully!


In [40]:
df.to_csv('Your Keywords Clustered.csv', index=False)
files.download("Your Keywords Clustered.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>